In [1]:
import numpy as np
import pandas as pd
import gc

import h2o
from h2o.automl import H2OAutoML

h2o.init(nthreads = 7,
        max_mem_size = '45g')

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 25.251-b08, mixed mode)
  Starting server from C:\Anaconda\envs\py36\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\anshch\AppData\Local\Temp\2\tmphfncfkem
  JVM stdout: C:\Users\anshch\AppData\Local\Temp\2\tmphfncfkem\h2o_anshch_started_from_python.out
  JVM stderr: C:\Users\anshch\AppData\Local\Temp\2\tmphfncfkem\h2o_anshch_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,12 secs
H2O cluster timezone:,Europe/Helsinki
H2O data parsing timezone:,UTC
H2O cluster version:,3.28.1.2
H2O cluster version age:,11 months and 19 days !!!
H2O cluster name:,H2O_from_python_anshch_v817gh
H2O cluster total nodes:,1
H2O cluster free memory:,40 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,7
H2O cluster status:,"accepting new members, healthy"


In [2]:
# Read data
data = pd.read_pickle(r'C:\Users\anshch\Documents\Demand_Forecast\Data\data_2.pkl')

test = pd.read_pickle(r'C:\Users\anshch\Documents\Demand_Forecast\Data\test.pkl')

In [4]:
# Select columns

data = data[[
    'date_block_num',
    'shop_id',
    'shop_category',
    'item_id',
    'item_cnt_month',
    # 'city_code',
    'item_category_id',
    'type_code',
    'subtype_code',
    'item_cnt_month_lag_1',
    'item_cnt_month_lag_2',
    'item_cnt_month_lag_3',
    #'item_cnt_month_lag_4',
    #'item_cnt_month_lag_5',
    #'item_cnt_month_lag_6',
    'item_cnt_month_lag_12',
    'date_avg_item_cnt_lag_1',
    'date_item_avg_item_cnt_lag_1',
    'date_item_avg_item_cnt_lag_2',
    'date_item_avg_item_cnt_lag_3',
    'date_item_avg_item_cnt_lag_6',
    'date_item_avg_item_cnt_lag_12',
    'date_shop_avg_item_cnt_lag_1',
    'date_shop_avg_item_cnt_lag_2',
    'date_shop_avg_item_cnt_lag_3',
    'date_shop_avg_item_cnt_lag_6',
    'date_shop_avg_item_cnt_lag_12',
    'date_cat_avg_item_cnt_lag_1',
    'date_shop_cat_avg_item_cnt_lag_1',
    'date_shop_type_avg_item_cnt_lag_1',
    'date_shop_subtype_avg_item_cnt_lag_1',
    'date_city_avg_item_cnt_lag_1',
    'date_item_city_avg_item_cnt_lag_1',
    'date_type_avg_item_cnt_lag_1',
    'date_subtype_avg_item_cnt_lag_1',
    'delta_price_lag',
    'month',
    'days',
    'item_shop_last_sale',
    'item_last_sale',
    'item_shop_first_sale',
    'item_first_sale',
    'city_coord_1',
    'city_coord_2',
    'country_part',
    'weeknd_count',
    'days_in_month'
]]

In [5]:
# Split data 
y = 'item_cnt_month'
X_train = data[data.date_block_num < 33]
X_valid = data[data.date_block_num == 33]
X_test = data[data.date_block_num == 34]

In [6]:
X_train['date_block_num'] = X_train['date_block_num'].astype(np.str)
X_valid['date_block_num'] = X_valid['date_block_num'].astype(np.str)
X_test['date_block_num'] = X_test['date_block_num'].astype(np.str)

C:\Anaconda\envs\py36\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Anaconda\envs\py36\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Anaconda\envs\py36\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

In [7]:
# Clean env
del data
gc.collect();

In [ ]:
x_train_hf = h2o.H2OFrame(X_train)
x_valid_hf = h2o.H2OFrame(X_valid)
x_test_hf = h2o.H2OFrame(X_test)

In [ ]:
# Run AutoML

aml = H2OAutoML(max_models = 5,
                seed=1)
aml.train(y=y, training_frame=x_train_hf, validation_frame = x_valid_hf)

In [ ]:
# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)

In [10]:
preds = aml.predict(x_test_hf)
preds = h2o.as_list(preds, use_pandas=True) 
Y_test = preds.clip(0, 20)

submission = pd.DataFrame({
    "ID": test.index, 
    "item_cnt_month": Y_test['predict'].values
})
submission.to_csv(r'C:\Users\anshch\Documents\Demand_Forecast\Results\h2o_aml_3.csv', index=False)

stackedensemble prediction progress: |████████████████████████████████████| 100%
